In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import csv
import sklearn

%matplotlib inline

/Users/nickryan/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [26]:
people = pd.read_csv('people.csv')
act_train = pd.read_csv('act_train.csv')
act_test = pd.read_csv('act_test.csv')
test_ids = act_test['activity_id']

In [15]:
act_train[1:50]

,people_id,activity_id,date,activity_category,char_1,char_2,char_3,char_4,char_5,char_6,char_7,char_8,char_9,char_10,outcome
1,ppl_100,act2_2434093,2022-09-27,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0
2,ppl_100,act2_3404049,2022-09-27,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0
3,ppl_100,act2_3651215,2023-08-04,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0
4,ppl_100,act2_4109017,2023-08-26,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0
5,ppl_100,act2_898576,2023-08-04,type 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1727,0
6,ppl_100002,act2_1233489,2022-11-23,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,1
7,ppl_100002,act2_1623405,2022-11-23,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,1
8,ppl_100003,act2_1111598,2023-02-07,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,1
9,ppl_100003,act2_1177453,2023-06-28,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,1
10,ppl_100003,act2_133509,2022-08-10,type 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 894,1


In [21]:
act_train.shape

(2197291, 15)

In [25]:
sum(act_train['outcome'].value_counts())

2197291

In [27]:
def preprocess_acts(data, train_set=True):
    
    # Getting rid of data feature for now
    data = data.drop(['date', 'activity_id'], axis=1)
    if(train_set):
        data = data.drop(['outcome'], axis=1)
    
    ## Split off _ from people_id
    data['people_id'] = data['people_id'].apply(lambda x: x.split('_')[1])
    data['people_id'] = pd.to_numeric(data['people_id']).astype(int)
    
    columns = list(data.columns)
    
    # Convert strings to ints
    for col in columns[1:]:
        data[col] = data[col].fillna('type 0')
        data[col] = data[col].apply(lambda x: x.split(' ')[1])
        data[col] = pd.to_numeric(data[col]).astype(int)
    return data

def preprocess_people(data):
    
    # TODO refactor this duplication
    data = data.drop(['date'], axis=1)
    data['people_id'] = data['people_id'].apply(lambda x: x.split('_')[1])
    data['people_id'] = pd.to_numeric(data['people_id']).astype(int)
    
    #  Values in the people df is Booleans and Strings    
    columns = list(data.columns)
    bools = columns[11:]
    strings = columns[1:11]
    
    for col in bools:
        data[col] = pd.to_numeric(data[col]).astype(int)        
    for col in strings:
        data[col] = data[col].fillna('type 0')
        data[col] = data[col].apply(lambda x: x.split(' ')[1])
        data[col] = pd.to_numeric(data[col]).astype(int)
    return data
# Preprocess each df
peeps = preprocess_people(people)
actions_train = preprocess_acts(act_train)
actions_test = preprocess_acts(act_test, train_set=False)
# Merege into a unified table


In [28]:
# Training 
features = actions_train.merge(peeps, how='left', on='people_id')
labels = act_train['outcome']

In [33]:
# Testing
test = actions_test.merge(peeps, how='left', on='people_id')

# Check it out...
features.sample(10)

,people_id,activity_category,char_1_x,char_2_x,char_3_x,char_4_x,char_5_x,char_6_x,char_7_x,char_8_x,...,char_29,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38
211382,129779,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1734950,380520,5,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,1,0,52
131670,115104,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2073996,78192,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,95
1916567,54699,3,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,38
1423112,335821,2,0,0,0,0,0,0,0,0,...,1,1,1,1,0,1,0,1,1,85
1388104,329334,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,14
268063,140143,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,91
1771707,386958,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,68
1761434,385243,4,0,0,0,0,0,0,0,0,...,0,0,1,1,0,1,0,1,1,90


In [45]:
## Split Training Data
from sklearn.cross_validation import train_test_split

num_test = 0.20
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=num_test, random_state=23)

## Out of box random forest
from sklearn.ensemble import RandomForestClassifier
#import xgboost as xgb
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.grid_search import GridSearchCV

clf = RandomForestClassifier(n_estimators=100, max_depth=5)
#clf = xgb.XGBClassifier(n_estimators=100, max_depth=5)
clf.fit(X_train, y_train)
## Training Predictions
proba = clf.predict_proba(X_test)
preds = proba[:,1]
score = roc_auc_score(y_test, preds)
print("Area under ROC {0}".format(score))
# Test Set Predictions
test_proba = clf.predict_proba(test)
test_preds = test_proba[:,1]

# Format for submission
output = pd.DataFrame({ 'activity_id' : test_ids, 'outcome': test_preds })
output.head()
output.to_csv('redhat.csv', index = False)

Area under ROC 0.918512908811
